<a href="https://colab.research.google.com/github/tomdkt/test-gpu-llama/blob/main/test_gpu_llama_cpp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!cat /etc/os-release
!nvcc --version
import torch
print(torch.cuda.is_available())

In [ ]:
# compile cuda llama-cpp-python. ~25 min
# !pip cache purge  # or pip cache remove llama-cpp-python
# !pip uninstall llama-cpp-python
# !CMAKE_ARGS="-DGGML_CUDA=on" FORCE_CMAKE=1 pip install llama-cpp-python

# download llama driver
# import os
# import llama_cpp

# lib_path = os.path.dirname(llama_cpp.__file__)
# zip_path = "/content/llama_cpp.zip"
# print(f"Zipping from: {lib_path}")
# !zip -r /content/llama_cpp.zip "$lib_path"

# # 3. (Opcional) download zip(~26mb)
# from google.colab import files
# files.download(zip_path)

from llama_cpp import llama_cpp
# gpu enable must be True
print(llama_cpp.llama_supports_gpu_offload())

In [2]:
# upload llama cpp driver
from google.colab import files
uploaded = files.upload()  # select `llama_cpp.zip`
!unzip -o llama_cpp.zip '*/llama_cpp/*' -d temp_llama
!cp -r temp_llama/usr/local/lib/python3.11/dist-packages/llama_cpp /usr/local/lib/python3.11/dist-packages/


!ls /usr/local/lib/python3.11/dist-packages/llama_cpp
!pip install diskcache
import sys
sys.path.append("/usr/local/lib/python3.11/dist-packages")

from llama_cpp import llama_cpp
# gpu enable must be True
print(llama_cpp.llama_supports_gpu_offload())


Archive:  llama_cpp.zip
  inflating: temp_llama/usr/local/lib/python3.11/dist-packages/llama_cpp/_logger.py  
  inflating: temp_llama/usr/local/lib/python3.11/dist-packages/llama_cpp/llama_grammar.py  
  inflating: temp_llama/usr/local/lib/python3.11/dist-packages/llama_cpp/_ggml.py  
  inflating: temp_llama/usr/local/lib/python3.11/dist-packages/llama_cpp/__pycache__/_logger.cpython-311.pyc  
  inflating: temp_llama/usr/local/lib/python3.11/dist-packages/llama_cpp/__pycache__/_ctypes_extensions.cpython-311.pyc  
  inflating: temp_llama/usr/local/lib/python3.11/dist-packages/llama_cpp/__pycache__/_ggml.cpython-311.pyc  
  inflating: temp_llama/usr/local/lib/python3.11/dist-packages/llama_cpp/__pycache__/llama.cpython-311.pyc  
  inflating: temp_llama/usr/local/lib/python3.11/dist-packages/llama_cpp/__pycache__/__init__.cpython-311.pyc  
  inflating: temp_llama/usr/local/lib/python3.11/dist-packages/llama_cpp/__pycache__/llama_chat_format.cpython-311.pyc  
  inflating: temp_llama/usr/lo

In [ ]:
from llama_cpp import Llama

llm = Llama.from_pretrained(
	repo_id="google/gemma-3-4b-it-qat-q4_0-gguf",
	filename="gemma-3-4b-it-q4_0.gguf",
  n_gpu_layers=-1
)

In [8]:
from IPython.display import display, HTML

response = llm.create_chat_completion(
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "oi!!! como vc está?"
                }
            ]
        }
    ],
    stream=True
)

# Accumulate the full response
full_response = ""

# Display initial HTML structure for dynamic updates
display(HTML("<b>Resposta:</b> <div id='response'></div><br><br><br>"))

for chunk in response:
    content = chunk["choices"][0]["delta"].get("content", "")
    #full_response += content

    # Update the HTML content with the latest response chunk
    # Using `.format()` instead of f-strings to avoid backslash issues
    display(HTML("<script>document.getElementById('response').innerHTML += '{}';</script>".format(content.replace('`', '\\`'))), display_id='response_update')


llama_perf_context_print:        load time =     328.89 ms
llama_perf_context_print: prompt eval time =     328.52 ms /    16 tokens (   20.53 ms per token,    48.70 tokens per second)
llama_perf_context_print:        eval time =     447.75 ms /    20 runs   (   22.39 ms per token,    44.67 tokens per second)
llama_perf_context_print:       total time =     857.31 ms /    36 tokens
